In [7]:
import pandas as pd
import altair as alt
from vega_datasets import data
# Create a Dataframe from a CSV
dataCSV = pd.read_csv('asianClinicsbyState.csv')

# Drop rows with any empty cells
dataCSV.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
dataCSV.head()

,id,state,Number of Clinic
0,1,Alabama,0
1,2,Alaska,0
2,4,Arizona,0
3,5,Arkansas,0
4,6,California,170


In [15]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [9]:
state_map = alt.topo_feature(data.us_10m.url, 'states')

click = alt.selection_multi(fields=['state'])

choropleth = (alt.Chart(state_map).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(dataCSV, 'id', ['Number of Clinic', 'state']))
.encode(
    color='Number of Clinic:Q',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    tooltip=['state:N', 'Number of Clinic:Q'])
.add_selection(click)
.project(type='albersUsa'))

bars = (
    alt.Chart(
        dataCSV.nlargest(15, 'Number of Clinic'),
        title='Top 15 States by Number of Clinic').mark_bar().encode(
    x='Number of Clinic',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    color='Number of Clinic',
    y=alt.Y('state', sort='x'),
    tooltip = ['state:N', 'Number of Clinic:Q'])
.add_selection(click))

# asianRestaurantsMap = alt.Chart(AsianRest).mark_circle().encode(
#     longitude='longitude:Q',
#     latitude='latitude:Q',
#     size=alt.value(20),
#     color=alt.value('#ff2400')
# #     tooltip=['name:N', 'address:N', 'city:N', 'state:N', 'categories:N']
# ).project(
#     "albersUsa"
# )

# restaurantMap = choropleth + asianRestaurantsMap
clinicMap = choropleth | bars
clinicMap

alt.HConcatChart(...)

In [10]:
clinicMap.save("AsianClinicMap.html", embed_options={"render": "svg"})